## 1. Loading the Dataset

This step reads the raw training and testing data into Pandas DataFrames, which are the fundamental structures for data manipulation in Python.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')


# Load the datasets
train_df = pd.read_csv('..\\data\\raw_data\\train.csv')
test_df = pd.read_csv('..\\data\\raw_data\\test.csv')

# Combine datasets for consistent preprocessing.
# I'll keep the distinction with an 'is_train' column for later splitting
train_df['is_train'] = 1
test_df['is_train'] = 0
combined_df = pd.concat([train_df, test_df], ignore_index=True)

print("DataFrames loaded successfully.")

### 2. Initial Data Exploration

Understanding the dataset's composition is crucial to planning the data cleaning strategy. Inspect the first few rows, check the overall dimensions, and review the data types.

In [ ]:
print("--- Combined DataFrame Head (First 5 Rows) ---")
print(combined_df.head())

print("\n--- Combined DataFrame Shape ---")
print(f"Rows: {combined_df.shape[0]}, Columns: {combined_df.shape[1]}")

print("\n--- Combined DataFrame Info (Data Types & Non-Null Counts) ---")
print(combined_df.info())

print("\n--- Descriptive Statistics (Numerical Columns) ---")
print(combined_df.describe().T)

# Check unique values in categorical columns to spot issues
print("\n--- Unique Values in Categorical Columns ---")
for col in ['Gender', 'Customer Type', 'Type of Travel', 'Class', 'satisfaction']:
    print(f"{col}: {combined_df[col].unique()}")

# Check for unexpected negative values in numerical columns (e.g., delays, distance, age)
# Age, Flight Distance, Departure Delay in Minutes, Arrival Delay in Minutes should all be >= 0.
numerical_check = combined_df[['Age', 'Flight Distance', 'Departure Delay in Minutes', 'Arrival Delay in Minutes']].min()
print("\n--- Minimum Values Check (Looking for unexpected negatives) ---")
print(numerical_check)

### 3. Handling Missing/Invalid Values

Missing data can skew the analysis and break models. It's essential to identify and address nulls and other obvious inconsistencies (like duplicates) early on.

In [ ]:
# Check for null values in the combined dataset
print("--- Missing Values Count ---")
missing_values = combined_df.isnull().sum()
missing_values = missing_values[missing_values > 0].sort_values(ascending=False)
print(missing_values)

#### Handling Missing Values

1. **`Arrival Delay in Minutes`**  
   This is the only column with missing values in the dataset, accounting for approximately **0.3%** of the total records.  
   To determine an appropriate imputation strategy, the distribution of arrival delays will be examined using a histogram.


In [ ]:
delay_data = combined_df["Arrival Delay in Minutes"].dropna() # Get only values without NANs.

# Calculate the mean, median and mode
data_mean = delay_data.mean()
data_median = delay_data.median()
data_mode = delay_data.mode().iloc[0] # take the first mode if multiple exist

# Draw histogram
plt.figure(figsize=(10,6))
plt.hist(
    delay_data,
    bins=200,
    density=False,
    alpha=0.7,
    color='skyblue',
    edgecolor='black'
)

# Add vertical lines for the statistics
plt.axvline(data_mean, color='red', linestyle='dashed', linewidth=0.5, 
            label=f'Mean: {data_mean:.2f} mins')
plt.axvline(data_median, color='green', linestyle='dashed', linewidth=0.5, 
            label=f'Median: {data_median:.2f} mins')
plt.axvline(data_mode, color='purple', linestyle='dashed', linewidth=0.5, 
            label=f'Mode: {data_mode:.2f} mins')

# Finalize the plot
plt.xlabel("Arrival Delay in Minutes")
plt.ylabel("Count of Flights")
plt.title(f'Distribution of Arrival Delays ')
plt.legend()
plt.grid(axis='y', alpha=0.5)
plt.tight_layout()

print(f"Mean: {data_mean} mins")
print(f"Median: {data_median} mins")
print(f"Mode: {data_mode} mins")

- **Mean:** 15.07 minutes  
- **Median:** 0.0 minutes  
- **Mode:** 0.0 minutes  

The histogram shows a **strong right-skewed distribution**, indicating that most flights are on time or experience minimal delays, while a smaller proportion of flights have large delays. Since both the **median and mode are 0**, missing values in the arrival delay column are **imputed with 0**.

In [ ]:
# Replaces all NaN values in Arrival Delay in Minutes with 0
combined_df["Arrival Delay in Minutes"] = combined_df["Arrival Delay in Minutes"].fillna(0)
print("Replace all NAN values in Arrival Delay in Minutes with zero")

#### Handling duplicates

In [ ]:

# --- Handling Duplicates ---

# Check for and drop duplicate rows (keeping the first occurrence)
print(f"\nNumber of duplicate rows before dropping: {combined_df.duplicated().sum()}")
combined_df.drop_duplicates(inplace=True)
print(f"Number of rows after dropping duplicates: {combined_df.shape[0]}")

### 4. Renaming Columns

Standardizing column names (e.g., using snake_case and removing spaces/special characters) improves readability and makes coding easier.

In [ ]:
# Create a dictionary for renaming columns to snake_case
new_columns = {
    'Customer Type': 'Customer_Type',
    'Type of Travel': 'Type_of_Travel',
    'Flight Distance': 'Flight_Distance',
    'Inflight wifi service': 'Inflight_wifi_service',
    'Departure/Arrival time convenient': 'Dep_Arr_Time_Convenient',
    'Ease of Online booking': 'Ease_of_Online_booking',
    'Gate location': 'Gate_Location',
    'Food and drink': 'Food_and_Drink',
    'Online boarding': 'Online_Boarding',
    'Seat comfort': 'Seat_Comfort',
    'Inflight entertainment': 'Inflight_Entertainment',
    'On-board service': 'On_board_Service',
    'Leg room service': 'Leg_Room_Service',
    'Baggage handling': 'Baggage_Handling',
    'Checkin service': 'Checkin_Service',
    'Inflight service': 'Inflight_Service',
    'Departure Delay in Minutes': 'Departure_Delay_in_Minutes',
    'Arrival Delay in Minutes': 'Arrival_Delay_in_Minutes'
}

combined_df.rename(columns=new_columns, inplace=True)

print("--- Sample of Renamed Columns ---")
print(combined_df.columns[1:10])

# Identify and remove the redundant index columns
# 'Unnamed: 0' is a remnant of an old index saved to CSV.
# 'id' is a unique identifier, not useful for modeling.
columns_to_drop = ['Unnamed: 0', 'id']

# Use errors='ignore' in case the column was already dropped in a previous run
combined_df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

print("Redundant columns 'Unnamed: 0' and 'id' have been successfully removed.")

### 5. Mapping Categorical Values

Some categorical variables are currently represented by numerical codes (0-5 scale for service ratings) or verbose strings. I'll map the target variable and address the numeric service ratings by ensuring they are recognized as categorical if a simple ordinal scale isn't sufficient for later analysis.

In [ ]:
# --- Mapping Target Variable ('satisfaction')  ---

# Map target variable to numerical (1 for satisfied, 0 for dissatisfied/neutral)
satisfaction_mapping = {
    'satisfied': 1,
    'neutral or dissatisfied': 0
}
combined_df['satisfaction'] = combined_df['satisfaction'].map(satisfaction_mapping)
print("\n'satisfaction' mapped to binary (1=satisfied, 0=dissatisfied).")
print(combined_df['satisfaction'].head())

# --- Handling Ordinal/Categorical Columns (Service Ratings) ---
# Service rating columns are ordinal (1–5), so they are kept as integers.

rating_cols = ['Inflight_wifi_service', 'Dep_Arr_Time_Convenient', 'Ease_of_Online_booking', 
               'Gate_Location', 'Food_and_Drink', 'Online_Boarding', 'Seat_Comfort', 
               'Inflight_Entertainment', 'On_board_Service', 'Leg_Room_Service', 
               'Baggage_Handling', 'Checkin_Service', 'Inflight_Service', 'Cleanliness']

# Convert to appropriate types (e.g., integer for rating, string for object types)
for col in rating_cols:
    combined_df[col] = combined_df[col].astype('int') 

# Convert other object columns to category type for efficiency
for col in ['Gender', 'Customer_Type', 'Type_of_Travel', 'Class']:
    combined_df[col] = combined_df[col].astype('category')
    
print("\nService rating columns coerced to integer. Key nominal columns converted to 'category' Dtype.")
print(combined_df.info())

### 6. Converting Data Types

This step ensures all columns have the most memory-efficient and appropriate data types before starting analysis.

In [ ]:
# Doing this for optimize the memory.
columns_list = ['Age', 'Flight_Distance', 'Departure_Delay_in_Minutes', 'Arrival_Delay_in_Minutes']
for col in columns_list:
    # Use 'Int64' to handle any remaining nullable integers gracefully, though I filled NaNs
    combined_df[col] = pd.to_numeric(combined_df[col], errors='coerce').round(0).astype('Int64')

# Here i used "errors='coerce'", so there can be NaNs due to unexpected characters. So check the NaNs again for safety.
print("Checking for NaNs introduced during the type change")
for col in columns_list:
    print(f"{col} NaN count: {combined_df[col].isnull().sum()}")
          
print("\n--- Final Data Types Check (Post-Conversion) ---")
print(combined_df.info())


### 7. Saving Cleaned Data

The final step is saving your cleaned data so you don't have to run the initial cleaning every time you return to the project.

In [ ]:
# Separate the processed combined data back into train and test sets
train_processed = combined_df[combined_df['is_train'] == 1].drop(columns=['is_train']).copy()
test_processed = combined_df[combined_df['is_train'] == 0].drop(columns=['is_train']).copy()

# Save the cleaned data to new CSV files
train_processed.to_csv('..\\data\\cleaned_data\\train_cleaned.csv', index=False)
test_processed.to_csv('..\\data\\cleaned_data\\test_cleaned.csv', index=False)

print("\nCleaned datasets saved as 'train_cleaned.csv' and 'test_cleaned.csv'.")
print("\n--- Final Cleaned Training Data Head ---")
print(train_processed.head())